# Sweep simulation

This notebook contains code to recreate Figure 2 in the paper. The tree sequence is also the ground truth dataset for certain supplementary figures.

In [1]:
import msprime

Let's declare some variables to use in the simulation. We specify that we want to draw `num_samples` sample nodes from a population size of `Ne`. The sequence we want to simulate is `seq_length` bases long, and the rate of recombination is specified with `recomb_rate`.

In [2]:
pop_size = 10_000
recomb_rate = 1e-8
mut_rate = 1e-8
num_samples = 300
seq_length = 5_000_000

We use `msprime` to simulate a tree sequence with the above parameters. We use the `SweepGenicSelection` function to specify the parameters of a selective sweep, with a beneficial allele situated in the middle of the sequence. The frequency of the allele when the sweep begins is indicated with `start_frequency` and the allele fixes in the population upon reaching a fequency `end_frequency`. `dt` is the small increment of time for stepping through the sweep phase of the model.

In [ ]:
sweep_model = msprime.SweepGenicSelection(
    position = seq_length / 2, 
    start_frequency = 0.0001, 
    end_frequency = 0.9999, 
    s = 0.25, 
    dt = 1e-6)

Next, we simulate the ancestry of the sampled genomes with `recomb_rate` to specify the recombination rate along the sequence. We use the selective sweep model defined above for the recent past, and the standard coalescent model for the deep past. We simulate mutations on the tree sequence at the rate of `mut_rate`, and write the tree sequence to file.

In [ ]:
models = [sweep_model, msprime.StandardCoalescent()]
ts = msprime.sim_ancestry(
    num_samples, 
    model = models, 
    population_size = pop_size, 
    sequence_length = seq_length, 
    recombination_rate = recomb_rate, random_seed=1234)
sim_ts = msprime.sim_mutations(ts, rate=mut_rate, random_seed=4321)
sim_ts.dump("true_topology.trees")

We can then run tsbrowse on these trees: <br>
`python -m tsbrowse preprocess true_topology.trees` <br>
`python -m tsbrowse serve --port 8090 true_topology.tsbrowse` <br>
and open the app in a web browser by typing: <br>
`localhost:8090` <br>
